Create video with plot visualization of force trajectory over time.
1. Video of top and side views with tracking of the pendulum rod tip.
2. Contact point of the stem along the rod- in side view.
3. visualization of force trajectory over time (with deflection, angle, and contact as well?).
4. maybe add an arrow of the Fxy direction in the top view video?

In [1]:
# import libraries

# generic
import sys
import numpy as np
import os,glob # for listing files in folder
import re # regular expressions
import pandas as pd
import scipy
import seaborn
import math as m
import cv2

from scipy.signal import savgol_filter
from scipy.spatial import distance as sci_distance
from scipy.stats import kruskal
import importlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import matplotlib.lines as mlines
import random
import itertools
from tqdm import tqdm
import pickle as pkl
import datetime
now = datetime.datetime.now()

# custom
import exp2funcs_clean # writch to updated exp2 functions
# sys.path.append('..')
import useful_functions as uf 

#general parameters
cmap = plt.get_cmap('viridis')
fs = 16 # standard font size for plots


print(os.getcwd())

importlib.reload(uf)
importlib.reload(exp2funcs_clean)

save_folder = r'C:\Users\Amir\Documents\PHD\Thesis\My Articles\0 - Flexible dynamic force measurement method via physical pendulum\Images'


hi
c:\Users\Amir\Documents\PHD\Python\GitHub\Amir_Repositories\Repo_article0
hi


In [2]:
# import data 
root_path = r'C:\Users\Amir\Documents\PHD\Experiments\Force Measurements'
basepath = root_path+r'\Exp2_Pendulum' # pendulum exp folder
excel= r'\Exp2_supplementary_measurements_events.csv'

# straw exp data
data_panda = pd.read_csv(basepath+excel)


# import tracked data, initiate variables
track_sup_path = r'\track_logs'
track_folder_path = basepath+track_sup_path

track_contact_path = r'\contact_track_logs'
contact_folder_path = basepath+track_contact_path

track_near_sup_path = r'\twine_init_logs'
track_near_sup_folder_path = basepath+track_near_sup_path

h5_path = r'\Measurements\root_stem_results'
h5_folder_path = basepath+h5_path

E_path = r'\Young_moduli'
E_folder_path = basepath+E_path

N_track = len(os.listdir(track_folder_path)) # get number of files in track folder
N_contact=len(os.listdir(contact_folder_path)) # get number of files in contact folder
N_tot = len(data_panda) # get number of lines in excel
N_E = len(os.listdir(E_folder_path))


#%% remove problematic events from raw data
delete_rows = [] # save rows to delete
problem_exp = [] # exp_num of problem events

for i in range(N_tot): # remove problem events and non-Helda events
    if data_panda.at[i,'problem']!='na' or data_panda.at[i,'Bean_Strain']!='Helda':
        delete_rows.append(i)
        problem_exp.append(data_panda.at[i,'Exp_num'])
N = N_tot - len(delete_rows) # modify num of rows

data_panda = data_panda.drop(data_panda.index[delete_rows]) # remove prob. events
data_panda = data_panda.reset_index() # redo index
#%% get misc. file lists

# get track files for support bottom coordinates
remove_chars = re.compile('[,_\.!?]') # what to remove from strings
track_dict = {} # save support track in dictionary by exp and events
i=0 # start with first track file
for file in glob.glob(os.path.join(track_folder_path, '*.txt')): # for each event:
    exp = int(re.findall('_\d{3,4}_',file)[0].replace('_','')) # find exp number (3-4 digits)
    event = int(re.findall('[0-9]',re.findall('_\d{1}\D',file)[0].replace('_',''))[0]) # find event number
    viewt = re.findall('(side{1}|top{1})',file)[0] #.replace('_','')
    track_dict[(exp,event,viewt)] = [file] # add new exp
    i+=1

# get track files for stem-support contact coordinates
contact_dict = {} # save support contact track in dictionary by exp and events
i=0 # start with first contact file
for file in glob.glob(os.path.join(contact_folder_path, '*.txt')): # for each event:
    exp = int(re.findall('_\d{3,4}_',file)[0].replace('_','')) # find exp number (3-4 digits)
    event = int(re.findall('_[0-9]_',file)[0].replace('_','')) # find event number
    contact_dict[(exp,event)] = [file] # add new exp
    i+=1

# get track files for 2 stem positions on either side of support
near_sup_track_dict = {}
i=0 # start with first stem_near_sup file
for file in glob.glob(os.path.join(track_near_sup_folder_path, '*.txt')): # for each event:
    exp = int(re.findall('_\d{3,4}_',file)[0].replace('_','')) # find exp number (3-4 digits)
    event = int(re.findall('_[0-9]_',file)[0].replace('_','')) # find event number
    near_sup_track_dict[(exp,event)] = [file] # add new exp
    i+=1

# get h5 files for stem near support
h5_dict = {}
i=0 # start with first h5 file
for file in glob.glob(os.path.join(h5_folder_path, '*.h5')): # for each event:
    exp = int(re.findall('interekt_\d{2,3}_',file)[0].split('_')[1]) # find exp number (3-4 digits)
    event = int(re.findall('e_[0-9]_',file)[0].split('_')[1]) # find event number
    start_frame = int(re.findall('_\d{2,5}-\d{2,5}',file)[0].replace('_','').split('-')[0]) # find start frame
    h5_dict[(exp,event,start_frame)] = [file] # add new exp
    i+=1

# get Young modulus files
E_dict = {}
for file in glob.glob(os.path.join(E_folder_path, '*.csv')):
    exp = int(re.findall('\d{2,3}',file)[0])
    E_dict[exp]=pd.read_csv(file,header=None)

#%%

In [3]:
#%% clear plants and events
plants = []
events = []
#%% populate plant and event instances
i = 0
N = len(data_panda)
for i in tqdm(range(N)): # N
    exp = int(re.findall('\d{3,4}',data_panda.at[i,'Exp_num'])[0]) # exp num
    view = data_panda.at[i,'View']  # side of top view
    if view == 'top':
        plants[-1].pix2cm_t = float(data_panda.at[i,'Top_pix2cm'])

    if i==0 or exp!=plants[-1].exp_num: # append new plant with data from pandas
        #basic data
        plants.append(exp2funcs_clean.Plant(data_panda,basepath,i,exp))
        # view dependent data
        plants[-1].view_data(data_panda,i)
        # circumnutation data
        plants[-1].cn_data(data_panda,i)
        #Youngs modulus by segment with avg
        plants[-1].getE(E_dict)


    event =  int(re.findall('_[0-9]',data_panda.at[i,
       'Exp_num'])[0].replace('_','')) # get event number


    # if this is the 1st event or the previous event_num is different from the current one:
    # add new event to list
    if len(events)==0 or events[-1].event_num != event or \
        events[-1].p.exp_num != exp:
        events.append(exp2funcs_clean.Event(plants[-1],data_panda,i))
    events[-1].event_num = event

    # view dependent data
    events[-1].view_data(data_panda,i,view)

    # get automated extraction of twine(decision) time
    events[-1].get_twine_time(exp,event,view,
                      h5_dict,near_sup_track_dict,50,track_dict,to_plot=0)


    # get track data, select decision period data, pix2cm,
    events[-1].event_base_calcs(view,track_dict,contact_dict)
    # calc
    events[-1].event_calc_variables(view)
    # print(f'i={i}, exp number {exp}, {event}') # print progress


  0%|          | 0/264 [00:00<?, ?it/s]

100%|██████████| 264/264 [00:11<00:00, 23.80it/s]


In [4]:
# create video of side and top view in split screen for a specific event
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas


def parallel_video(images1, coords1, images2, coords2, fps=2, fixed_height=300):
    """
    Display two sets of images side by side with markers in a resizable window.
    
    Parameters:
        images1 (list): First set of images (file paths or image arrays).
        coords1 (list): Coordinates for the first set of images (list of (x, y) tuples).
        images2 (list): Second set of images (file paths or image arrays).
        coords2 (list): Coordinates for the second set of images (list of (x, y) tuples).
        fps (int): Frames per second for image display.
        fixed_height (int): Fixed height of the final window.
    """
    if len(images1) != len(coords1) or len(images2) != len(coords2):
        raise ValueError("The number of images and coordinates must match for both sets.")
    if len(images1) != len(images2):
        raise ValueError("Both sets of images must have the same length.")
    
    delay = 1 / fps  # Delay in seconds between frames
    
    # Create a resizable window
    window_name = "Image Viewer"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    
    for idx, (img1, coord1, img2, coord2) in enumerate(zip(images1, coords1, images2, coords2)):
        # Load the images if they are file paths, otherwise assume they are arrays
        if isinstance(img1, str):
            img1 = cv2.imread(img1)
        if isinstance(img2, str):
            img2 = cv2.imread(img2)
        
        # Ensure both images are in the same color space (BGR)
        if len(img1.shape) < 3:
            img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
        if len(img2.shape) < 3:
            img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
        
        # Draw markers on each image
        if coord1:
            cv2.drawMarker(img1, (int(coord1[0]), int(coord1[1])), color=(0, 0, 255), 
                           markerType=cv2.MARKER_CROSS, thickness=4, markerSize=25)
        if coord2:
            cv2.drawMarker(img2, (int(coord2[0]), int(coord2[1])), color=(0, 0, 255), 
                           markerType=cv2.MARKER_TILTED_CROSS, thickness=4, markerSize=25)
        # Keep previous marker positions on each subsequent image
        for prev_coord1 in coords1[:idx]:
            cv2.drawMarker(img1, (int(prev_coord1[0]), int(prev_coord1[1])), color=(0, 255, 0), 
                   markerType=cv2.MARKER_CROSS, thickness=2, markerSize=15)
        for prev_coord2 in coords2[:idx]:
            cv2.drawMarker(img2, (int(prev_coord2[0]), int(prev_coord2[1])), color=(0, 255, 0), 
                   markerType=cv2.MARKER_TILTED_CROSS, thickness=2, markerSize=15)
        # Make both images the same height for horizontal concatenation
        height = max(img1.shape[0], img2.shape[0])
        img1 = cv2.resize(img1, (int(img1.shape[1] * height / img1.shape[0]), height))
        img2 = cv2.resize(img2, (int(img2.shape[1] * height / img2.shape[0]), height))
        
        # Concatenate images horizontally
        combined_image = cv2.hconcat([img1, img2])
        
        # Scale the combined image to have a fixed height
        scale_ratio = fixed_height / combined_image.shape[0]
        scaled_width = int(combined_image.shape[1] * scale_ratio)
        resized_image = cv2.resize(combined_image, (scaled_width, fixed_height))
        
        # Set the window size to the resized image dimensions
        cv2.resizeWindow(window_name, resized_image.shape[1], resized_image.shape[0])
        
        # Display the resized concatenated image in the window
       
        # Calculate the position to center the window
        window_x = 1400
        window_y = 50
        
        # Move the window to the center of the screen
        cv2.moveWindow(window_name, window_x, window_y)
        
        cv2.imshow(window_name, resized_image)
        
        # Wait for the specified delay or until a key is pressed
        if idx == len(images1) - 1:
            # Keep the last image displayed until the user closes the window
            print("Final image displayed. Close the window to end.")
            cv2.waitKey(0)
        else:
            cv2.waitKey(int(delay * 1000))  # Convert delay to milliseconds
    
    # Close all OpenCV windows
    cv2.destroyAllWindows()


def display_images_with_graph(images1, coords1, images2, coords2, graph_data, fps=2, fixed_height=600, save_video=False, output_path="output_video.mp4"):
    """
    Display two sets of images side by side with a live-updating graph below them, 
    and optionally save the sequence as a video.

    Parameters:
        images1 (list): First set of images (file paths or image arrays).
        coords1 (list): Coordinates for the first set of images (list of (x, y) tuples).
        images2 (list): Second set of images (file paths or image arrays).
        coords2 (list): Coordinates for the second set of images (list of (x, y) tuples).
        graph_data (list): List of (x, y) tuples for graph plotting.
        fps (int): Frames per second for image display.
        fixed_height (int): Fixed height of the final display window.
        save_video (bool): Whether to save the sequence as a video.
        output_path (str): Path to save the video (if enabled).
    """
    if len(images1) != len(coords1) or len(images2) != len(coords2) or len(images1) != len(graph_data):
        raise ValueError("The number of images, coordinates, and graph data points must match.")
    
    delay = 1 / fps  # Delay in seconds between frames
    
    # Create a resizable window
    window_name = "Image Viewer"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    
    # Initialize video writer if saving the video
    video_writer = None
    if save_video:
        frame_width = 2 * fixed_height  # Assuming width proportional to height
        frame_height = int(fixed_height + fixed_height * 0.6)  # Extra height for graph
        video_writer = cv2.VideoWriter(
            output_path,
            cv2.VideoWriter_fourcc(*"mp4v"),
            fps,
            (frame_width, frame_height)
        )
    
    for idx, (img1, coord1, img2, coord2, (x_vals, y_vals)) in enumerate(zip(images1, coords1, images2, coords2, graph_data)):
        # Load the images if they are file paths, otherwise assume they are arrays
        if isinstance(img1, str):
            img1 = cv2.imread(img1)
        if isinstance(img2, str):
            img2 = cv2.imread(img2)
        
        # Ensure both images are in the same color space (BGR)
        if len(img1.shape) < 3:
            img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
        if len(img2.shape) < 3:
            img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
        
        # Draw markers on each image
        if coord1:
            cv2.drawMarker(img1, (int(coord1[0]), int(coord1[1])), color=(0, 0, 255), 
                           markerType=cv2.MARKER_CROSS, thickness=3, markerSize=50)
        if coord2:
            cv2.drawMarker(img2, (int(coord2[0]), int(coord2[1])), color=(0, 0, 255), 
                           markerType=cv2.MARKER_TILTED_CROSS, thickness=3, markerSize=50)
        # Keep previous marker positions on each subsequent image
        for prev_coord1 in coords1[:idx]:
            cv2.drawMarker(img1, (int(prev_coord1[0]), int(prev_coord1[1])), color=(255, 0, 0), 
                   markerType=cv2.MARKER_CROSS, thickness=2, markerSize=40)
        for prev_coord2 in coords2[:idx]:
            cv2.drawMarker(img2, (int(prev_coord2[0]), int(prev_coord2[1])), color=(255, 0, 0), 
                   markerType=cv2.MARKER_TILTED_CROSS, thickness=2, markerSize=40)
            
        # Resize images to the fixed height
        height = fixed_height
        img1 = cv2.resize(img1, (int(img1.shape[1] * height / img1.shape[0]), height))
        img2 = cv2.resize(img2, (int(img2.shape[1] * height / img2.shape[0]), height))
        
        # Concatenate images horizontally
        combined_image = cv2.hconcat([img1, img2])
        
        # Create the graph as an image
        fig, ax = plt.subplots(figsize=(4, 3))
        max_x = max(graph_data, key=lambda x: x[0])[0]
        max_y = max(graph_data, key=lambda x: x[1])[1]
        ax.set_xlim([0, max_x * 1.1])
        ax.set_ylim([0, max_y * 1.1])
        ax.plot(x_vals, y_vals, marker="o", color="blue", linewidth=2)
        # for prev_xy in graph_data[:idx]:
        for prev_x_vals, prev_y_vals in graph_data[:idx]:
            ax.plot(prev_x_vals, prev_y_vals,  marker="o", color="black", linewidth=2)
        ax.set_xlabel("t(sec)")
        ax.set_ylabel("F(N)")
        ax.grid(True)
        canvas = FigureCanvas(fig)
        canvas.draw()
        graph_img = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
        graph_img = graph_img.reshape(canvas.get_width_height()[::-1] + (3,))
        plt.close(fig)
        
        # Resize the graph to match the combined image width
        graph_height = int(combined_image.shape[0] * 0.5)  # Scale graph height relative to images
        graph_img = cv2.resize(graph_img, (combined_image.shape[1], graph_height))
        
        # Concatenate graph below the images
        final_frame = cv2.vconcat([combined_image, graph_img])
        
        # Resize window to match final frame dimensions
        cv2.resizeWindow(window_name, final_frame.shape[1], final_frame.shape[0])
        
        # Calculate the position to center the window
        window_x = 100
        window_y = 50
        
        # Move the window to the center of the screen
        cv2.moveWindow(window_name, window_x, window_y)
        
        # Display the final frame
        cv2.imshow(window_name, final_frame)
        
        # Write to video if saving
        if save_video and video_writer:
            video_writer.write(cv2.cvtColor(final_frame, cv2.COLOR_RGB2BGR))
        
        # Wait for the specified delay or until a key is pressed
        if idx == len(images1) - 1:
            # Keep the last frame displayed until the user closes the window
            print("Final frame displayed. Close the window to end.")
            cv2.waitKey(0)
        else:
            cv2.waitKey(int(delay * 1000))  # Convert delay to milliseconds
    
    # Release the video writer
    if save_video and video_writer:
        video_writer.release()
    
    # Close all OpenCV windows
    cv2.destroyAllWindows()

def display_images_with_graph2(
    images1, coords1, images2, coords2, graph_data, fps=2, fixed_height=600, save_video=False, output_path="output_video.mp4"
):
    """
    Display two sets of images side by side with a live-updating graph below them,
    and optionally save the sequence as a video.

    Parameters:
        images1 (list): First set of images (file paths or image arrays).
        coords1 (list): Coordinates for the first set of images (list of (x, y) tuples).
        images2 (list): Second set of images (file paths or image arrays).
        coords2 (list): Coordinates for the second set of images (list of (x, y) tuples).
        graph_data (list): List of (x, y) tuples for graph plotting.
        fps (int): Frames per second for image display.
        fixed_height (int): Fixed height of the final display window.
        save_video (bool): Whether to save the sequence as a video.
        output_path (str): Path to save the video (if enabled).
    """
    if len(images1) != len(coords1) or len(images2) != len(coords2) or len(images1) != len(graph_data):
        raise ValueError("The number of images, coordinates, and graph data points must match.")

    delay = 1 / fps  # Delay in seconds between frames

    # Create a resizable window
    window_name = "Image Viewer"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

    # Initialize video writer if saving the video
    video_writer = None
    frame_width = 2 * fixed_height  # Width proportional to height
    frame_height = int(fixed_height + 200)  # Extra height for graph
    if save_video:
        video_writer = cv2.VideoWriter(
            output_path,
            cv2.VideoWriter_fourcc(*"mp4v"),
            fps,
            (frame_width, frame_height),
        )

    for idx, (img1, coord1, img2, coord2, (x_vals, y_vals)) in enumerate(zip(images1, coords1, images2, coords2, graph_data)):
        # Load the images if they are file paths, otherwise assume they are arrays
        if isinstance(img1, str):
            img1 = cv2.imread(img1)
        if isinstance(img2, str):
            img2 = cv2.imread(img2)

        # Ensure both images are in the same color space (BGR)
        if len(img1.shape) < 3:
            img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
        if len(img2.shape) < 3:
            img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

        # Draw markers on each image
        if coord1:
            cv2.drawMarker(img1, (int(coord1[0]), int(coord1[1])), color=(0, 0, 255),
                           markerType=cv2.MARKER_CROSS, thickness=3, markerSize=45)
        if coord2:
            cv2.drawMarker(img2, (int(coord2[0]), int(coord2[1])), color=(0, 0, 255),
                           markerType=cv2.MARKER_TILTED_CROSS, thickness=3, markerSize=45)
        # Keep previous marker positions on each subsequent image
        for prev_coord1 in coords1[:idx]:
            cv2.drawMarker(img1, (int(prev_coord1[0]), int(prev_coord1[1])), color=(255, 0, 0), 
                   markerType=cv2.MARKER_CROSS, thickness=2, markerSize=40)
        for prev_coord2 in coords2[:idx]:
            cv2.drawMarker(img2, (int(prev_coord2[0]), int(prev_coord2[1])), color=(255, 0, 0), 
                   markerType=cv2.MARKER_TILTED_CROSS, thickness=2, markerSize=40)
            
        # Resize images to the fixed height
        height = fixed_height
        img1 = cv2.resize(img1, (int(img1.shape[1] * height / img1.shape[0]), height))
        img2 = cv2.resize(img2, (int(img2.shape[1] * height / img2.shape[0]), height))

        # Concatenate images horizontally
        combined_image = cv2.hconcat([img1, img2])

        # Create the graph as an image
        fig, ax = plt.subplots(figsize=(8, 2))  # Fixed aspect ratio
        ax.plot(x_vals, y_vals, marker="o", color="blue", linewidth=2)
        for prev_x_vals, prev_y_vals in graph_data[:idx]:
            ax.plot(prev_x_vals, prev_y_vals, marker="o", color="black", linewidth=2)
        ax.set_xlabel("Time (s)")
        ax.set_ylabel("Force (N)")
        max_x = max(graph_data, key=lambda x: x[0])[0]
        max_y = max(graph_data, key=lambda x: x[1])[1]
        ax.set_xlim([0, max_x * 1.1])
        ax.set_ylim([0, max_y * 1.1])
        ax.grid(True)
        canvas = FigureCanvas(fig)
        canvas.draw()
        graph_img = np.frombuffer(canvas.tostring_rgb(), dtype="uint8")
        graph_img = graph_img.reshape(canvas.get_width_height()[::-1] + (3,))
        plt.close(fig)

        # Resize the graph to match the width of the combined image
        graph_img = cv2.resize(graph_img, (combined_image.shape[1], 200))
        # Convert graph image to RGB
        graph_img = cv2.cvtColor(graph_img, cv2.COLOR_BGR2RGB)
        # Concatenate graph below the images
        final_frame = cv2.vconcat([combined_image, graph_img])

        # Convert to BGR for OpenCV and ensure dimensions match and return to RBG
        # final_frame_bgr = cv2.cvtColor(final_frame, cv2.COLOR_RGB2BGR)
        # final_frame = cv2.cvtColor(final_frame, cv2.COLOR_BGR2RGB)

        # Resize window to match final frame dimensions
        # cv2.resizeWindow(window_name, final_frame_bgr.shape[1], final_frame_bgr.shape[0])
        cv2.resizeWindow(window_name, final_frame.shape[1], final_frame.shape[0])

        # Display the final frame
        # cv2.imshow(window_name, final_frame_bgr)
        cv2.imshow(window_name, final_frame)
       
        if video_writer is None and save_video:
            # frame_width, frame_height = final_frame_bgr.shape[1], final_frame_bgr.shape[0]
            frame_width, frame_height = final_frame.shape[1], final_frame.shape[0]
            video_writer = cv2.VideoWriter(
                output_path,
                cv2.VideoWriter_fourcc(*"mp4v"),
                fps,
                (frame_width, frame_height),
            )

        if save_video and video_writer:
            # final_frame_resized = cv2.resize(final_frame_bgr, (frame_width, frame_height))
            final_frame_resized = cv2.resize(final_frame, (frame_width, frame_height))
            video_writer.write(final_frame_resized)

        # cv2.resizeWindow(window_name, final_frame_bgr.shape[1], final_frame_bgr.shape[0])
        cv2.resizeWindow(window_name, final_frame.shape[1], final_frame.shape[0])
        # cv2.imshow(window_name, final_frame_bgr)
        cv2.imshow(window_name, final_frame)

        if idx == len(images1) - 1:
            print("Final frame displayed. Close the window to end.")
            cv2.waitKey(0)
        else:
            cv2.waitKey(int(delay * 1000))

    if save_video and video_writer:
        video_writer.release()

    cv2.destroyAllWindows()


In [ ]:
#%%  Define the experiment and event numbers
# exp_num = 30
# event_num = 1
# find the event corresponding to exp_num and event_num:

i = 70 # 40 (simple) , 20,120 (bump from dist2tip), i=70->event 79_2
event = events[i]
print(event.p.exp_num,event.event_num)
print(event.frm0_top,event.frm_dec_top)

# start video from initial contact frame for both views

# Define paths to the video frames
side_view_path = r"C:\Users\Amir\Documents\PHD\Thesis\My Articles\0 - Flexible dynamic force measurement method via physical pendulum\Data\sample trajectory\79_2_side"
top_view_path = r"C:\Users\Amir\Documents\PHD\Thesis\My Articles\0 - Flexible dynamic force measurement method via physical pendulum\Data\sample trajectory\79_2_top"
vid_path = r"C:\Users\Amir\Documents\PHD\Thesis\My Articles\0 - Flexible dynamic force measurement method via physical pendulum\Data\sample trajectory"

top_view_files = [os.path.join(top_view_path, f) for f in os.listdir(top_view_path) if os.path.isfile(os.path.join(top_view_path, f))]
side_view_files = [os.path.join(side_view_path, f) for f in os.listdir(side_view_path) if os.path.isfile(os.path.join(side_view_path, f))]                                                                    

# Define the number of frames
fr = 50
fr = min(fr,len(top_view_files))

# top images and tracked coordiantes
top_images = top_view_files[event.frm0_top:event.frm_dec_top][:fr]
dec_x_track_top = event.x_track_top0[event.frm0_top:event.frm_dec_top][:fr]
dec_y_track_top = event.y_track_top0[event.frm0_top:event.frm_dec_top][:fr]
top_xy_coor = list(zip(dec_x_track_top,dec_y_track_top))

# side images and tracked coordiantes
side_images = side_view_files[event.frm0_side:event.frm_dec_side][:fr]
dec_x_track_side = event.x_track_side0[event.frm0_side:event.frm_dec_side][:fr]
dec_y_track_side = event.z_track_side0[event.frm0_side:event.frm_dec_side][:fr]
side_xz_coor = list(zip(dec_x_track_side,dec_y_track_side))
t = event.timer[event.frm0_top:event.frm_dec_top][:fr]-event.timer[event.frm0_top]
# force data
force_data = list(zip(t,event.F_bean[event.frm0_top:event.frm_dec_top][:fr]))

# parallel_video(top_images, top_xy_coor,side_images, side_xz_coor, fps=2)
display_images_with_graph2(top_images, top_xy_coor,side_images, side_xz_coor, graph_data=force_data , fps=5,
                          fixed_height=300, save_video=False, output_path=vid_path+"\\side_top_f(t)_video.mp4")


79 2
11 196


C:\Users\Amir\AppData\Local\Temp\ipykernel_26152\1567752453.py:314: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  graph_img = np.frombuffer(canvas.tostring_rgb(), dtype="uint8")


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1473: error: (-27:Null pointer) NULL window: 'Image Viewer' in function 'cvResizeWindow'
